<a href="https://colab.research.google.com/github/btrentini/data_science/blob/master/US_Census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#US Census 

⚠ **Run notebook:** In the menu above click "Runtime > Run all" or press $CTRL + F9$ 

## Task Summary

In summary, this is what this notebook does:

- Extracts and Prepares dataset from http://thomasdata.s3.amazonaws.com/ds/us_census_full.zip
- EDA and Feature Engineering
- Learns, for a dataset formed by vectors $\{x_n\}_{n=0}^N$, N the number of training examples, a model for the label $y_n$.  
- Predicts, given an a vector $x_n=\{x_n^0, x_n^1, ..., x_n^k\}$, $k$ the number of features, and the model learnt, if target variable, $y$ (salary), is greater than or equal to \$50,000 per year, yielding a prediction $\hat{y_n}$ for each training example $x_n$.
- Test different models and validate on test set the cross-entropy loss $L = - \sum_n y_n \, \,log \, \hat{y_n} + (1- y_n)log(1-\hat{y_n})$ that yields the lowerst error for the pair $(y, \hat{y})$ 



## Task Info

The following link lets you download an archive containing an “exercise” US Census dataset: http://thomasdata.s3.amazonaws.com/ds/us_census_full.zip
This US Census dataset contains detailed but anonymized information for approximately 300,000 people.

>The archive contains 3 files: 
* A large training file (csv)
* Another test file (csv)
* A metadata file (txt) describing the columns of the two csv files (identical for both)

> **The goal** of this exercise is to model the information contained in the last column (42nd), i.e., whether a person makes more or less than $50,000 per year, from the information contained in the other columns. The exercise here consists of modeling a binary variable.

> Work with Python (or R) to carry out the following steps:
*  Load the train and test files.
* Perform an exploratory analysis on the data and create some relevant visualisations.
* Clean, preprocess, and engineer features in the training data, with the aim of building a data set that a model will perform well on.
* Create a model using these features to predict whether a person earns more or less than $50,000 per year. Here, the idea is for you to test a few different models, and see whether there are any techniques you can apply to improve performance over your first results.
* Choose the model that appears to have the highest performance based on a comparison between reality (the 42nd variable) and the model’s prediction. 
* Apply your model to the test file and measure its real performance on it (same method as above).

>The goal of this exercise is not to create the best or the purest model, but rather to describe the steps you took to accomplish it.
Explain areas that may have been the most challenging for you.
>Find clear insights on the profiles of the people that make more than $50,000 / year. For example, which variables seem to be the most correlated with this phenomenon?
>Finally, you push your code on GitHub to share it with me, or send it via email.

>Once again, the goal of this exercise is not to solve this problem, but rather to spend a few hours on it and to thoroughly explain your approach.

## Metadata Info

**From the metadata (see below how this was obtained):**


> This data was extracted from the census bureau database found at
>http://www.census.gov/ftp/pub/DES/www/welcome.html

>Donor: Terran Lane and Ronny Kohavi
       Data Mining and Visualization
       Silicon Graphics.
       e-mail: terran@ecn.purdue.edu, ronnyk@sgi.com for questions.


>The data was split into train/test in approximately $2/3$, $1/3$ proportions using MineSet's MIndUtil mineset-to-mlc.

>**Prediction task** is to determine the income level for the person represented by the record.  Incomes have been binned at the $50K level to present a binary classification problem, much like the original UCI/ADULT database.  The goal field of this data, however, was drawn from the "total person income" field rather than the "adjusted gross income" and may, therefore, behave differently than the orginal ADULT goal field.
>More information detailing the meaning of the attributes can be found in http://www.bls.census.gov/cps/cpsmain.htm

# Setup

## Check GPU configs

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
!pip install wget

## Imports

In [ ]:
# System utils
import sys
import os
import zipfile
import time

# Parallelism
from joblib import Parallel, delayed

# Some classic data science stuff
import numpy as np; 
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
%matplotlib inline 

# Math & Stats
import math 
import scipy.stats as stats
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor    

# Machine Learning libs
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, plot_importance

# Scikit learn utils
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



In [ ]:
# Styling
sns.set_style("ticks", {"xtick.major.size": 11, "ytick.major.size": 11})
sns.set_palette("inferno")
sns.set(font_scale = 2)
figsize=(23, 15)
pal='inferno'
div = "="*72

# Build datasets

## Download & Extract

In [ ]:
!wget "http://thomasdata.s3.amazonaws.com/ds/us_census_full.zip"

In [ ]:
!ls -1

In [ ]:
# Define helper to load
local_zip =os.path.join('/content', 'us_census_full.zip')

# Unzip Train Set into temporary path
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
!ls -1 /tmp

In [ ]:
train     = '/tmp/us_census_full/census_income_learn.csv'
test      = '/tmp/us_census_full/census_income_test.csv'
metadata  = '/tmp/us_census_full/census_income_metadata.txt'

In [ ]:
# Let's see what's in the metadata
!fold -w 130 -s $metadata

In [ ]:
# Check if there's a header in the train file
!head -2 $train

In [ ]:
# Check if there's a header in the test file
!head -2 $test

In [ ]:
%time df  = pd.read_csv(train, header=None)
%time t_df = pd.read_csv(test, header=None) 

In [ ]:
# Check
df.head(5)

## A trick from metadata for column names
 This will help us a lot during EDA. The metada contains useful information about columns, values and their properties. I can use this file to name columns and later on this will give us the option to address the dataframe by column names, which might be handy in many cases

In [ ]:
!tail -42 $metadata

In [ ]:
'''
From the above we can see that the last 42 rows are the column names
We can use this info to improve our datasets and help us eith EDA

Besides, the metada tell us to ignore '|_instance_weight'the 24th record
'''

# We will beed a list to append to...
cols = []

# Save metadata last 42 rows
column_names = !tail -42 $metadata

# Remove the record to be ignored '|_instance_weight'
column_names.pop(24)        

# Build column helper
for col in column_names:
  record = col.split(":")[0].replace(" ","_")
  cols.append(record)

# Add target variable's column not listed in metadata
cols.append("target")

# Insert column names into dataframes
df.columns = cols
t_df.columns = cols

# Voila!
df.head(5)

In [ ]:
df.target.value_counts()

**Note:** Dataset quite unbalanced...So we have two options:

- Either do Downscaling / Upscaling; Or
- Create a method that accounts for this imbalance in the proper way

More on this later...

**Transform year columns** as its obvious from Metadata it shouldn't be continuous

In [ ]:
df.head(5)

## Categorise data 

Lets convert objects into categorical values so that we can manipulate them numerically

In [ ]:
obj = df.select_dtypes(['object'])
df[obj.columns] = obj.apply(lambda x: x.str.strip().replace(" ",""))

In [ ]:
df_bkp = df.copy()
t_df = t_df.copy()

for col in df.columns:
  if df[col].dtype == 'object':
    df[col] = df[col].str.strip().replace(" ","").astype('category').cat.codes
    t_df[col] = df[col].str.strip().replace(" ","").astype('category').cat.codes

df.dtypes

## Are there null / undefined values?

There are a few records marked with "$?$" but we will treat this as a category... So I wonder if there are columns with null values or "na" values


In [ ]:
df.target

In [ ]:
null_columns=df.columns[df.isnull().any()]
if len(null_columns) > 0:
  print(df[null_columns].isnull().sum())
else:
  print("No null columns")

na_columns=df.columns[df.isna().any()]
if len(na_columns) > 0:
  print(df[na_columns].isna().sum())
else:
  print("No NA columns")

## Are there duplicates?

In [ ]:
if df.duplicated().sum() > 0:
  print(df.duplicated().sum(), " dupicated records found and dropped")
  df.drop_duplicates(inplace=True)

# Exploratory Data Analysis

## Feature Importance

Start with a simple light GBM for estimating feature importance

In [ ]:
model = LGBMClassifier(learning_rate=0.01, num_leaves= 33, random_state=42)
model.fit(df.iloc[:,0:41], df.iloc[:,0])

In [ ]:
fig, ax = plt.subplots(figsize=(figsize))
ax = sns.barplot(y=model.feature_importances_, x=df.iloc[:,0:41].columns, palette=pal)
plt.xticks(rotation=90)

## Correlation

Will help us understand risks of colinearity and some features that we can get scrap. We will only consider features with moderate (>0.5), strong (>0.7) and very strong (>0.9) correlation in absolute values

In [ ]:
correlation = df.drop(['target'], axis=1).corr()
mask = np.triu(correlation)
with sns.axes_style("white"):
  f, ax = plt.subplots(figsize=(23,23))
  #ax = sns.heatmap(correlation[(correlation > 0.5) | (correlation < -0.5)],
  ax = sns.heatmap(correlation,
              square=True,
              vmax=1.0,
              vmin=-1.0,
              center=0.0,
              cmap="coolwarm",
              linecolor='white',
              linestyle = '--',
              rasterized=False,
              edgecolor='white',
              capstyle='projecting',
              linewidth=2,
              mask=mask,
              annot=False, 
              fmt=".2f",
              robust=True,
              cbar=True,
              cbar_kws={"location": "top",
                        'use_gridspec': False,
                        "label": "Correlation Coefficient",
                        'shrink': 0.8})

### Variance Inflation Factor(VIF)
From wikipedia: VIF is the quotient of the variance in a model with multiple terms by the variance of a model with one term alone. It quantifies the severity of multicollinearity in an ordinary least squares regression analysis. It provides an index that measures how much the variance (the square of the estimate's standard deviation) of an estimated regression coefficient is increased because of collinearity.

> Multicollinearity is a common phenomenon in high‐dimensional settings, in which two or more predictor variables are highly correlated [Zhao et al, 2020]( https://doi.org/10.1002/sta4.272)

This is a bit of a controversial topic and "tule of thumb" tresholds are dangerous as pointed in [A Caution Regarding Rules of Thumb
for Variance Inflation Factors](https://www.researchgate.net/profile/Robert_Obrien8/publication/226005307_A_Caution_Regarding_Rules_of_Thumb_for_Variance_Inflation_Factors/links/54d0f2620cf298d656695641/A-Caution-Regarding-Rules-of-Thumb-for-Variance-Inflation-Factors.pdf). The lowest VIF valuee is 1. Anything beyond 10 is extreme. Most people choose either $3$, $4$ or $5$ as treshold. We will go with $4$.

❌ We have $k=41$ features. With large $k$ this would be innapropriate but there are solusions like the one proposed in [Zhao et al, 2020]( https://doi.org/10.1002/sta4.272)


In [ ]:
X = add_constant(df.iloc[:,0:41])
multico_indx=pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)

In [ ]:
sns.distplot(multico_indx, rug=True)

In [ ]:
df.iloc[:,np.where((multico_indx[1:] < 4)==True)[0][:].tolist()].dtypes

In [ ]:
df = df.iloc[:,np.where((multico_indx[1:] < 4)==True)[0][:].tolist()]
df['target'] = df_bkp['target'].astype('category').cat.codes

In [ ]:
df.head(5)

### Post VIF check feature importance

In [ ]:
model = LGBMClassifier(learning_rate=0.01, num_leaves= 33, random_state=42)
model.fit(df.iloc[:,:-1], df.iloc[:,-1])

fig, ax = plt.subplots(figsize=(figsize))
ax = sns.barplot(y=model.feature_importances_, x=df.iloc[:,:-1].columns, palette=pal)
plt.xticks(rotation=90)

### Post VIF check correlation

In [ ]:
correlation = df.drop(['target'], axis=1).corr()
mask = np.triu(correlation)
with sns.axes_style("white"):
  f, ax = plt.subplots(figsize=(23,23))
  #ax = sns.heatmap(correlation[(correlation > 0.5) | (correlation < -0.5)],
  ax = sns.heatmap(correlation,
              square=True,
              vmax=1.0,
              vmin=-1.0,
              center=0.0,
              cmap="coolwarm",
              linecolor='white',
              linestyle = '--',
              rasterized=False,
              edgecolor='white',
              capstyle='projecting',
              linewidth=2,
              mask=mask,
              annot=False, 
              fmt=".2f",
              robust=True,
              cbar=True,
              cbar_kws={"location": "top",
                        'use_gridspec': False,
                        "label": "Correlation Coefficient",
                        'shrink': 0.8})

It Looks like country of birth from father and mother are moderately correlated with citizenship. Since we can (almost) always derive someone's citizenship from their parents' place of birth and assuming parents' nationality do not play an imediate factor in someone's wages (setting apart deeper social analyses) and considering neither parent's origin appears as a crucial feature in our Light GBM model, we will remove these features to simplify our model further

In [ ]:
df.drop(columns=['country_of_birth_father'], inplace=True)
df.drop(columns=['country_of_birth_mother'], inplace=True)

There's a moderate linear relationship between the industry of a person with its ocupation and the number of persons worked for employer. Althought these shouldn't affect our model aggressively, we can infer one by the other (for instance: we assume that someone's occupation is highly related to its industry and that its industry can tell information about the number of people per employer. Think about Social Media industry versus Manufacuring plant.)

In [ ]:
df.drop(columns=['detailed_occupation_recode'], inplace=True)
df.drop(columns=['num_persons_worked_for_employer'], inplace=True)

In the USA, similar to the UK and other countries like Switzerland, someone's marital status will change the tax code. So we can infer the former by the later and can get rid of another feature with moderate correlation

In [ ]:
df.drop(columns=['marital_stat'], inplace=True)

In [ ]:
correlation = df.drop(['target'], axis=1).corr()
mask = np.triu(correlation)
with sns.axes_style("white"):
  f, ax = plt.subplots(figsize=(23,23))
  ax = sns.heatmap(correlation[(correlation > 0.5) | (correlation < -0.5)],
  #ax = sns.heatmap(correlation,
              square=True,
              vmax=1.0,
              vmin=-1.0,
              center=0.0,
              cmap="coolwarm",
              linecolor='white',
              linestyle = '--',
              rasterized=False,
              edgecolor='white',
              capstyle='projecting',
              linewidth=2,
              mask=mask,
              annot=False, 
              fmt=".2f",
              robust=True,
              cbar=True,
              cbar_kws={"location": "top",
                        'use_gridspec': False,
                        "label": "Correlation Coefficient",
                        'shrink': 0.8})

**NICE!** *So no correlations greater than 0.5 or less than -0.5. Let's plot the rest:*

In [ ]:
correlation = df.drop(['target'], axis=1).corr()
mask = np.triu(correlation)
with sns.axes_style("white"):
  f, ax = plt.subplots(figsize=(23,23))
  #ax = sns.heatmap(correlation[(correlation > 0.5) | (correlation < -0.5)],
  ax = sns.heatmap(correlation,
              square=True,
              vmax=1.0,
              vmin=-1.0,
              center=0.0,
              cmap="coolwarm",
              linecolor='white',
              linestyle = '--',
              rasterized=False,
              edgecolor='white',
              capstyle='projecting',
              linewidth=2,
              mask=mask,
              annot=True,
              fmt=".1f",
              robust=True,
              cbar=True,
              annot_kws={"size": 16},
              cbar_kws={"location": "top",
                        'use_gridspec': False,
                        "label": "Correlation Coefficient",
                        'shrink': 0.8})

## Further EDA
Let's explore our data a little further. Then perhaps come up with a simple hypothesis to test and a prediction model

In [ ]:
# Lets use the non categorized dataset for some analyses
eda = df_bkp.loc[:,df.columns.tolist()]

## Distplot

Warm up: do we have a population that is equally distirbuted in terms of age? No. We can see that 50% of the population is 30 years old or younger. We have the next 30% at circa 60 years old or less. We can also confirm that by fitting a loggama distribution to the data (second chart, black line)

In [ ]:
# Bimodal data, 50% of population less than 30yo
fig, ax = plt.subplots(2, 1, figsize=figsize)
kwargs = {'cumulative': True}

g1 = sns.distplot(eda['age'], hist_kws=kwargs, kde_kws=kwargs, ax=ax[0], axlabel=None,
                  fit=stats.gamma, bins=10)
g2 = sns.distplot(eda['age'], rug=True, ax=ax[1], fit=stats.loggamma)

How about the target by age? We can see that 50% of those earning $< 50$K are around 30 years old or young. On the other hand, 50% of those earning $>50$K are older than 45 years old.

In [ ]:
fig, ax = plt.subplots(figsize=(figsize))
g = sns.boxplot(y='age', x='target', data=eda, palette=pal)
plt.xticks(rotation=0)

# Hypothesis Test 1: Constructors vs Educators

**Let's distill the following hypothesis:**

"There's a better chance of seeing *Construction* workers among the top earners in comparison to *Education* workers. 

>$H_0: \, \, $ Equal chance of *Construction* workers earning  +50K as *Education* workers   ($\theta=0.5$)    
>$H_1: \, \, $ There's a better chance of seeing *Construction* workers earning  +50K in comparison to *Education* workers ($\theta > 0.5$)  


Where $H_0$ is our null hypothesis and $H_1$ is our alternative hypothesis, our test will be conducted at a significance level of 5%


🚀 Let's ignore EDA and answer this with the right level of confidence in a statistical way



In [ ]:
eda

In [ ]:
# Let's first compute the proportion of construction workers and educators among the top earners
test_statistic = eda[(eda.major_industry_code=="Construction") & (eda.target!='-50000.') ].target.count()
complement = eda[(eda.major_industry_code=="Education") & (eda.target!='-50000.') ].target.count()

# How many observations do we have in total
N = test_statistic+complement
print("Total people earning +50K = ", N, test_statistic+complement==N)
print("--- construction =", test_statistic)
print("--- education =", complement)

**To simplify, we can model this as a binomial distribution** (seeing construction workers in the high earning group is considered success)

*Level = 5%*:  Decreasing this will decrease our critical region (region where we fail
 o reject the null hypothesis if the p-value falls within)

*p=0.5:*  We want to test if the propability of seeing constructors and educators are the same

In [ ]:
level=0.05
p=0.5

In [ ]:
print(f"Within the 50K+ target we have seen {N} constructors and educators")
print(f"{test_statistic} of them are in the Construction Sector ({round(test_statistic/N,2)*100}%)\n")
print(f"\nH0 : The proportion of consutrction workers as top earners is the same as educators (param = {p})")
print(f"H1 : There are fewer construction workers earning 50K+ than educators (param < {p})\n")

%time p_value = np.sum([stats.binom_test(x, N, p=0.5, alternative='greater') for x in range(test_statistic)])
print(f"p-value = {p_value} -->  for seeing {test_statistic} construction workers or less")

print("\n", div)
if p_value < level:
  print('''\nTHEREFORE We reject the null hypothesis H0.
  There is enough evidence at 5% level of significance to suggest that 
  >> there are more constructors than educators in the top earners\n''')
else:
  print('''\nTHEREFORE We FAIL to reject the null hypotehesis H0.
  There is enough evidence at 5% level of significance to suggest that 
  >> constructors and educators are balanced as top earners\n''')
print(div)


# Hypothesis 2: What if we compare constructors against Agriculture workers? 

**Let's distill the following hypothesis:**

"There's a better chance of seeing *Construction* workers among the top earners in comparison to *Agriculture* workers. 

>$H_0: \, \, $ Equal chance of *Construction* workers earning  +50K as *Agriculture* workers   ($\theta=0.5$)    
>$H_1: \, \, $ There's a better chance of seeing *Construction* workers earning  +50K in comparison to *Agriculture* workers ($\theta > 0.5$)  


Where $H_0$ is our null hypothesis and $H_1$ is our alternative hypothesis, our test will be conducted at a significance level of 5%


🚀 Let's ignore EDA and answer this with the right level of confidence in a statistical way



In [ ]:
# Let's first compute the proportion of construction workers and educators among the top earners
test_statistic = eda[(eda.major_industry_code=="Construction") & (eda.target!=' -50000.') ].major_industry_code.value_counts()[0]
complement = eda[(eda.major_industry_code=="Agriculture") & (eda.target!='-50000.') ].major_industry_code.value_counts()[0]

# How many observations do we have in total
N = test_statistic+complement
print("Total people earning +50K = ", N, test_statistic+complement==N)
print("--- construction =", test_statistic)
print("--- Agriculture =", complement)

In [ ]:
level=0.05
p=0.5

In [ ]:
print(f"Within the 50K+ target we have seen {N} constructors and educators")
print(f"{test_statistic} of them are in the Construction Sector ({round(test_statistic/N,2)*100}%)\n")
print(f"\nH0 : The proportion of consutrction workers as top earners is the same as educators (param = {p})")
print(f"H1 : There are fewer construction workers earning 50K+ than educators (param < {p})\n")

%time p_value = np.sum([stats.binom_test(x, N, p=0.5, alternative='greater') for x in range(test_statistic)])
print(f"p-value = {p_value} -->  for seeing {test_statistic} construction workers or less")

print("\n", div)
if p_value < level:
  print('''\nTHEREFORE We reject the null hypothesis H0.
  There is enough evidence at 5% level of significance to suggest that 
  >> there are more constructors than educators in the top earners\n''')
else:
  print('''\nTHEREFORE We FAIL to reject the null hypotehesis H0.
  There is enough evidence at 5% level of significance to suggest that 
  >> constructors and educators are balanced among top earners\n''')
print(div)


## Split Train into Train (70%) & Validation (30%)

In [ ]:
df # Is this the categorized one?

In [ ]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

xtrain, xvalid, ytrain, yvalid = train_test_split(X, y, test_size=0.3, random_state=42)

## XGBOOST
Please note this is not the Scikit-learn API but the python API: https://xgboost.readthedocs.io/en/latest/python/python_intro.html which provides better support for GPUs

In [ ]:
# instantiate params
params = {}

# general params
general_params = {'silent': 1,
                  'lambda': 0.02,
                  'learning_rate': 0.01,
                  'max_depth': 7}
params.update(general_params)

# booster params
n_gpus = 1  # change this to -1 to use all GPUs available or 0 to use the CPU
booster_params = {}

if n_gpus != 0:
    booster_params['tree_method'] = 'gpu_hist'
    booster_params['n_gpus'] = n_gpus   
params.update(booster_params)

# learning task params
learning_task_params = {}
learning_task_params['eval_metric'] = ['logloss', 'rmse']
learning_task_params['objective'] = 'reg:logistic'

params.update(learning_task_params)

In [ ]:
model = XGBClassifier(**params, n_estimators=1000, early_stopping_rounds=10, verbose=2)

In [ ]:
print("Train...")
%time model.fit(xtrain, ytrain)

print("\nPredict (Validation)...")
%time y_pred = model.predict(xvalid)
predictions=[round(value) for value in y_pred]

In [ ]:
accuracy = accuracy_score(yvalid, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

rmse = np.sqrt(mean_squared_error(yvalid, y_pred))
print("RMSE: %f" % (rmse))

In [ ]:
cv_results = xgb.cv(**params, as_pandas=True)

In [ ]:
print("\nPredict (Test)...")

y_pred=model.predict(t_df.iloc[:,0:41])
predictions=[round(value) for value in y_pred]
accuracy = accuracy_score(t_df.iloc[:,41], predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))